## Train GPT on addition

Train a GPT model on a dedicated addition dataset to see if a Transformer can learn to add.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
from torch.utils.data import Dataset

class AdditionDataset(Dataset):
    """
    Returns addition problems of up to some number of digits in the inputs. Recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
    encoding will simply be the n-digit first number, n-digit second number, 
    and (n+1)-digit result, all simply concatenated together. Because each addition
    problem is so structured, there is no need to bother the model with encoding
    +, =, or other tokens. Each possible sequence has the same length, and simply
    contains the raw digits of the addition problem.
    
    As a few examples, the 2-digit problems:
    - 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
    - 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
    etc.
    
    We will also only train GPT on the final (n+1)-digits because the first
    two n-digits are always assumed to be given. So when we give GPT an exam later,
    we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
    to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5]
    in 3 sequential steps.
    
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        idx = self.ixes[idx]
        nd = 10**self.ndigit
        a = idx // nd
        b = idx %  nd
        c = a + b
        render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028" 
        dix = [int(s) for s in render] # convert each character to its token index
        # x will be input to GPT and y will be the associated expected outputs
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long) # predict the next token in the sequence
        y[:self.ndigit*2-1] = -100 # we will only train in the output locations. -100 will mask loss to zero
        return x, y


In [5]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset = AdditionDataset(ndigit=ndigit, split='train')
test_dataset = AdditionDataset(ndigit=ndigit, split='test')

In [6]:
train_dataset[0] # sample a training instance just to see what one raw example looks like

(tensor([4, 7, 1, 7, 0, 6]), tensor([-100, -100, -100,    0,    6,    4]))

In [7]:
from mingpt.model_delight import DelightConfig, DelightModel

# initialize a baby GPT model
mconf = DelightConfig(train_dataset.vocab_size, train_dataset.block_size, 
                  n_layer=4, n_head=1, n_embd=128)
model = DelightModel(mconf)

08/22/2020 03:43:06 - INFO - mingpt.model_delight -   number of parameters: 4.393480e+05


In [8]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=100, batch_size=512, learning_rate=6e-4, 
                      weight_decay=0, betas=(0.9, 0.98), grad_norm_clip=0.,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset)*(ndigit+1),
                      num_workers=4)
trainer = Trainer(model, train_dataset, test_dataset, tconf)
trainer.train()

epoch 1 iter 17: train loss 1.80311. lr 5.994512e-04: 100%|██████████| 18/18 [00:25<00:00,  1.42s/it]
08/22/2020 03:43:33 - INFO - mingpt.trainer -   test loss: 1.779751
epoch 2 iter 17: train loss 1.59130. lr 5.977197e-04: 100%|██████████| 18/18 [00:33<00:00,  1.84s/it]
08/22/2020 03:44:07 - INFO - mingpt.trainer -   test loss: 1.544506
epoch 3 iter 17: train loss 1.47722. lr 5.948114e-04: 100%|██████████| 18/18 [00:26<00:00,  1.50s/it]
08/22/2020 03:44:35 - INFO - mingpt.trainer -   test loss: 1.416624
epoch 4 iter 17: train loss 1.38523. lr 5.907379e-04: 100%|██████████| 18/18 [00:23<00:00,  1.29s/it]
08/22/2020 03:45:00 - INFO - mingpt.trainer -   test loss: 1.298190
epoch 5 iter 17: train loss 1.25611. lr 5.855153e-04: 100%|██████████| 18/18 [00:30<00:00,  1.69s/it]
08/22/2020 03:45:31 - INFO - mingpt.trainer -   test loss: 1.187733
epoch 6 iter 17: train loss 1.24354. lr 5.791641e-04: 100%|██████████| 18/18 [00:24<00:00,  1.35s/it]
08/22/2020 03:45:57 - INFO - mingpt.trainer -   

epoch 49 iter 17: train loss 0.74242. lr 6.000000e-05: 100%|██████████| 18/18 [00:16<00:00,  1.12it/s]
08/22/2020 03:59:53 - INFO - mingpt.trainer -   test loss: 0.585240
epoch 50 iter 17: train loss 0.75502. lr 6.000000e-05: 100%|██████████| 18/18 [00:17<00:00,  1.01it/s]
08/22/2020 04:00:12 - INFO - mingpt.trainer -   test loss: 0.579192
epoch 51 iter 17: train loss 0.75266. lr 6.000000e-05: 100%|██████████| 18/18 [00:16<00:00,  1.09it/s]
08/22/2020 04:00:30 - INFO - mingpt.trainer -   test loss: 0.580730
epoch 52 iter 17: train loss 0.76075. lr 6.000000e-05: 100%|██████████| 18/18 [00:17<00:00,  1.03it/s]
08/22/2020 04:00:48 - INFO - mingpt.trainer -   test loss: 0.570095
epoch 53 iter 17: train loss 0.74971. lr 6.000000e-05: 100%|██████████| 18/18 [00:12<00:00,  1.41it/s]
08/22/2020 04:01:01 - INFO - mingpt.trainer -   test loss: 0.561761
epoch 54 iter 17: train loss 0.71352. lr 6.000000e-05: 100%|██████████| 18/18 [00:14<00:00,  1.25it/s]
08/22/2020 04:01:16 - INFO - mingpt.traine

epoch 97 iter 17: train loss 0.14973. lr 5.948114e-04: 100%|██████████| 18/18 [00:12<00:00,  1.38it/s]
08/22/2020 04:12:27 - INFO - mingpt.trainer -   test loss: 0.025640
epoch 98 iter 17: train loss 0.30458. lr 5.977197e-04: 100%|██████████| 18/18 [00:13<00:00,  1.36it/s]
08/22/2020 04:12:41 - INFO - mingpt.trainer -   test loss: 0.034791
epoch 99 iter 17: train loss 0.18942. lr 5.994512e-04: 100%|██████████| 18/18 [00:13<00:00,  1.38it/s]
08/22/2020 04:12:54 - INFO - mingpt.trainer -   test loss: 0.038843
epoch 100 iter 17: train loss 0.18235. lr 5.999991e-04: 100%|██████████| 18/18 [00:12<00:00,  1.40it/s]
08/22/2020 04:13:08 - INFO - mingpt.trainer -   test loss: 0.020059


In [9]:
# now let's give the trained model an addition exam
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import sample

def give_exam(dataset, batch_size=32, max_batches=-1):
    
    results = []
    loader = DataLoader(dataset, batch_size=batch_size)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        d1d2 = x[:, :ndigit*2]
        d1d2d3 = sample(model, d1d2, ndigit+1)
        d3 = d1d2d3[:, -(ndigit+1):]
        factors = torch.tensor([[10**i for i in range(ndigit+1)][::-1]]).to(trainer.device)
        # decode the integers from individual digits
        d1i = (d1d2[:,:ndigit] * factors[:,1:]).sum(1)
        d2i = (d1d2[:,ndigit:ndigit*2] * factors[:,1:]).sum(1)
        d3i_pred = (d3 * factors).sum(1)
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))
        
        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [10]:
# training set: how well did we memorize?
give_exam(train_dataset, batch_size=1024, max_batches=10)

GPT claims that 002 + 002 = 014 (gt is 004; NOPE)
GPT claims that 030 + 034 = 074 (gt is 064; NOPE)
GPT claims that 002 + 001 = 013 (gt is 003; NOPE)
GPT claims that 002 + 003 = 015 (gt is 005; NOPE)
GPT claims that 032 + 032 = 074 (gt is 064; NOPE)
GPT claims that 028 + 071 = 109 (gt is 099; NOPE)
GPT claims that 030 + 033 = 073 (gt is 063; NOPE)
GPT claims that 030 + 032 = 072 (gt is 062; NOPE)
GPT claims that 001 + 001 = 012 (gt is 002; NOPE)
GPT claims that 007 + 002 = 019 (gt is 009; NOPE)
GPT claims that 000 + 001 = 011 (gt is 001; NOPE)
GPT claims that 006 + 003 = 019 (gt is 009; NOPE)
GPT claims that 003 + 002 = 015 (gt is 005; NOPE)
GPT claims that 004 + 001 = 015 (gt is 005; NOPE)
GPT claims that 004 + 002 = 016 (gt is 006; NOPE)
GPT claims that 008 + 011 = 029 (gt is 019; NOPE)
GPT claims that 002 + 000 = 012 (gt is 002; NOPE)
GPT claims that 003 + 001 = 014 (gt is 004; NOPE)
GPT claims that 000 + 002 = 012 (gt is 002; NOPE)
GPT claims that 001 + 002 = 013 (gt is 003; NOPE)


In [11]:
# test set: how well did we generalize?
give_exam(test_dataset, batch_size=1024, max_batches=-1)

GPT claims that 049 + 040 = 099 (gt is 089; NOPE)
GPT claims that 030 + 035 = 075 (gt is 065; NOPE)
GPT claims that 001 + 000 = 011 (gt is 001; NOPE)
final score: 997/1000 = 99.70% correct


In [12]:
# well that's amusing... our model learned everything except 55 + 45